In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('AXISX_resample.csv')
df_np = np.array(df)
print(df)

          chip    wire parameter  segment     value
0        CHIP1  WIRE10     AXISX        1  1.873603
1        CHIP1  WIRE10     AXISX        1  2.015115
2        CHIP1  WIRE10     AXISX        1  2.033436
3        CHIP1  WIRE10     AXISX        1  1.987634
4        CHIP1  WIRE10     AXISX        1  1.932988
...        ...     ...       ...      ...       ...
480063  CHIP10   WIRE9     AXISX        7  0.598421
480064  CHIP10   WIRE9     AXISX        7  0.549144
480065  CHIP10   WIRE9     AXISX        7  0.520084
480066  CHIP10   WIRE9     AXISX        7  0.539036
480067  CHIP10   WIRE9     AXISX        7  0.611056

[480068 rows x 5 columns]


In [3]:
# AXISX - 1 (CHIP WIRE 구분 X)
cls1 = df_np[df_np[:,3] == 1, :]
cls2 = df_np[df_np[:, 3] == 2, :]
cls3 = df_np[df_np[:, 3] == 3, :]
cls4 = df_np[df_np[:, 3] == 4, :]
cls5 = df_np[df_np[:, 3] == 5, :]
cls6 = df_np[df_np[:, 3] == 6, :]
cls7 = df_np[df_np[:, 3] == 7, :]

In [4]:
def Labeling(label, ran):
    lst = []
    for i in range(ran):
        num = i + 1
        s = '{0}{1}'.format(label, num)
        lst.append(s)
    return lst
chip = Labeling('CHIP', 10)
wire = Labeling('WIRE', 19)
print(chip)
print(wire)

['CHIP1', 'CHIP2', 'CHIP3', 'CHIP4', 'CHIP5', 'CHIP6', 'CHIP7', 'CHIP8', 'CHIP9', 'CHIP10']
['WIRE1', 'WIRE2', 'WIRE3', 'WIRE4', 'WIRE5', 'WIRE6', 'WIRE7', 'WIRE8', 'WIRE9', 'WIRE10', 'WIRE11', 'WIRE12', 'WIRE13', 'WIRE14', 'WIRE15', 'WIRE16', 'WIRE17', 'WIRE18', 'WIRE19']


In [9]:
# 1번 class에서 chip 10개로 나눔
## 목표 :총 10개의 ( chip1, chip2, ... chip10) 을 기준으로 나누어진 배열을 저장한 리스트 생성
## 3차원 이고 각 배열마다 행이 다름 (10, 가변길이, 5)
def split_out_chip(df, labels):
    cls_sp = []
    for label in labels:
        c = df[df[:, 0] == label, :]
        cls_sp.append(c)
    cls_sp = np.array(cls_sp)## DataFrame 형식으로 변경
    return cls_sp
#############################################################
# 1번 class chip 10개로 분리된 리스트 다시 19개 wire로 분리
## 목표: 총 190개의 array를 가지는 리스트로 생성
## 3차원 이며 (190, 가변길이, 5)

def split_out_wire(df, labels):
    cls_sp = []
    for i in range(df.shape[0]):
        dfdf = df[i]
        for label in labels:
            c = dfdf[dfdf[:, 1] == label, :]
            cls_sp.append(c)
    cls_sp = np.array(cls_sp)## DataFrame 형식으로 변경
    return cls_sp
############################################################
# chip wire 분류 함수 합치기
def split_out(df, chips, wires):
    # CHIP 분류
    cls_c = split_out_chip(df, chips)
    # WIRE 분류
    cls_c_w = split_out_wire(cls_c, wires)
    return cls_c_w
cls1_c_w = split_out(cls1, chip, wire)
cls2_c_w = split_out(cls2, chip, wire)
cls3_c_w = split_out(cls3, chip, wire)
cls4_c_w = split_out(cls4, chip, wire)
cls5_c_w = split_out(cls5, chip, wire)
cls6_c_w = split_out(cls6, chip, wire)
cls7_c_w = split_out(cls7, chip, wire)

In [10]:
print(cls1_c_w)

[array([['CHIP1', 'WIRE1', 'AXISX', 1, 1.38621],
       ['CHIP1', 'WIRE1', 'AXISX', 1, 1.4740229999999999],
       ['CHIP1', 'WIRE1', 'AXISX', 1, 1.41527],
       ...,
       ['CHIP1', 'WIRE1', 'AXISX', 1, 3.33989],
       ['CHIP1', 'WIRE1', 'AXISX', 1, 3.3869550000000004],
       ['CHIP1', 'WIRE1', 'AXISX', 1, 3.4131730000000005]], dtype=object)
 array([['CHIP1', 'WIRE2', 'AXISX', 1, 1.8916080000000002],
       ['CHIP1', 'WIRE2', 'AXISX', 1, 2.247598],
       ['CHIP1', 'WIRE2', 'AXISX', 1, 2.229909],
       ['CHIP1', 'WIRE2', 'AXISX', 1, 2.130725],
       ['CHIP1', 'WIRE2', 'AXISX', 1, 2.003743],
       ['CHIP1', 'WIRE2', 'AXISX', 1, 1.9724720000000002],
       ['CHIP1', 'WIRE2', 'AXISX', 1, 2.019853],
       ['CHIP1', 'WIRE2', 'AXISX', 1, 2.034067],
       ['CHIP1', 'WIRE2', 'AXISX', 1, 1.9601529999999998],
       ['CHIP1', 'WIRE2', 'AXISX', 1, 1.8019],
       ['CHIP1', 'WIRE2', 'AXISX', 1, 1.602584],
       ['CHIP1', 'WIRE2', 'AXISX', 1, 1.5510959999999998],
       ['CHIP1', 'WIRE2'

## LSTM 을 사용한 feature 값 추출

**Method**

1. 각 시계열 데이터 셋을 Train dataset & Test dataset 으로 나눈다. 비율은 약 9:1
    - validation dataset경우 : cross_validation 기법 / 초기 분할
2. Train/Test dataset을 N=time-step 으로 x, y dataset으로 각각 나눈다.
    - X_train, Y_train shape = (train 개수, N, 1) (train 개수, 1)
    - X_test, Y_test shape = (test 개수, N, 1) (test 개수, 1)
3. 모델을 쌓고 학습시킨다.
    - 예상되는 결과 : 타임스텝에 따른 